<a href="https://colab.research.google.com/github/corinnakeum1/DS2002Project1/blob/main/DS2002Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Data Ingestion

  - Eggs data from an API
  - Milk data from local CSV

In [2]:
import pandas as pd
# Load the CSV files into pandas DataFrames
Egg_Price = pd.read_csv('/content/egg.csv')

# Find missing values in GHG_totals_per_country.csv
missing_totals = Egg_Price[Egg_Price.isnull().any(axis=1)]


In [3]:
# rename columns in the data frame called : "APU0000708111" to be "price", and "observation_date" to be "date"
Egg_Price = Egg_Price.rename(columns={"APU0000708111": "price", "observation_date": "date"})

#print df to see changes
Egg_Price

,date,price
0,1980-01-01,0.879
1,1980-02-01,0.774
2,1980-03-01,0.812
3,1980-04-01,0.797
4,1980-05-01,0.737
...,...,...
537,2024-10-01,3.370
538,2024-11-01,3.649
539,2024-12-01,4.146
540,2025-01-01,4.953


In [4]:
# Finding missing values in Egg_Price.csv
missing_values = Egg_Price[Egg_Price.isnull().any(axis=1)]

# Print the rows with missing values
missing_values


,date,price


In [5]:
import requests

API_KEY = "48c6378d20063bf646888ca8893518aa"  # Register and replace with your key
SERIES_ID = "APU0000709112"  # The ID for your dataset
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

params = {
    "series_id": SERIES_ID,
    "api_key": API_KEY,
    "file_type": "json",
}

# Fetch data from FRED API
response = requests.get(BASE_URL, params=params)
data = response.json()

# Extract observations
observations = data.get("observations", [])

# Convert to DataFrame
df = pd.DataFrame(observations)

# Save to CSV
milkcsv = "milkdata.csv"
df.to_csv(milkcsv, index=False)

print(f"Data saved to {milkcsv}")


Data saved to milkdata.csv


2. Merging data into one dataset
  - Modifying dataset by removing columns
  - Data analysis with summary statistics

In [12]:
#Summarizing injested data before processing
  #for eggs
num_rows, num_cols = Egg_Price.shape
print(f"Number of records: {num_rows}")
print(f"Number of columns: {num_cols}")
  #for milk
milkcsv = pd.read_csv("milkdata.csv")
num_rows, num_cols = milkcsv.shape
print(f"Number of records: {num_rows}")
print(f"Number of columns: {num_cols}")

#read in the csv files
eggdata = Egg_Price
milkdata = pd.read_csv('milkdata.csv')

#merge on the date columns
egg_milk_data = pd.merge(eggdata, milkdata, on='date', how='inner')

#rename the columns
egg_milk_data.rename(columns={
    #'observation_date': 'Date',
    'price': 'Egg_price',
    'realtime_start': 'Data_grab_date',
    'realtime_end': 'Data_grab_date2',
    'date': 'Date2',
    'value': 'Milk_price'}, inplace=True)

#removing unessesary columns
egg_milk_data.drop(columns=['Data_grab_date','Data_grab_date2','Date2'], inplace=True)

#save to new CSV
egg_milk_data.to_csv('egg_milk.csv', index=False)

print(egg_milk_data.head())

#data analysis of summary statistics
egg_milk_data.describe()

#summary of transformed data
num_rows, num_cols = egg_milk_data.shape
print(f"Number of records: {num_rows}")
print(f"Number of columns: {num_cols}")


Number of records: 542
Number of columns: 2
Number of records: 356
Number of columns: 4
   Egg_price  Milk_price
0      0.879       2.477
1      0.984       2.482
2      0.956       2.459
3      0.981       2.473
4      1.037       2.493
Number of records: 356
Number of columns: 2


Allow users to convert between data types

In [13]:
#JSON to CSV, CSV to JSON, JSON to SQL, CSV to SQL
import sqlite3
import json

#function converts JSON to CSV
def json_to_csv(JSON, CSV):
    csv_df = pd.read_json(JSON)
    csv_df.to_csv(CSV, index=False)
    print(f"Converted {JSON} to {CSV}")

#function converts CSV to JSON again
def csv_to_json(CSV, JSON):
    json_df = pd.read_csv(CSV)
    json_df.to_json(JSON, orient='records', indent=4)
    print(f"Converted {CSV} to {JSON}")

#function converts JSON to SQL
def json_to_sql(JSON, SQL):
    with open(JSON, 'r') as f:
        data = json.load(f)
    sql_df = pd.DataFrame(data)
    conn = sqlite3.connect('egg_milk.db')
    sql_df.to_sql(SQL, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Converted {JSON} to SQL table '{SQL}' in egg_milk.db")

#finally to convert csv to sql
def csv_to_sql(CSV, SQL):
    sql_df = pd.read_csv(CSV)
    conn = sqlite3.connect('egg_milk.db')
    sql_df.to_sql(SQL, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Converted {CSV} to SQL table '{SQL}' in egg_milk.db")

#user puts in the desired function and data to convert between formats
def format_converter(function,data,new_name):
  if desired_function == 'json_to_csv':
    json_to_csv(data,new_name)
  if desired_function == 'csv_to_json':
    csv_to_json(data,new_name)
  if desired_function == 'json_to_sql':
    json_to_sql(data,new_name)
  if desired_function == 'csv_to_sql':
    csv_to_sql(data,new_name)
  print (f'Converted {data}')

# example: format_converter(csv_to_sql,'egg_milk.csv','egg_milk_sql')

#here the user would use the format_converter function
desired_function = input() #what you want to convert to
desired_data = input() #input name of the JSON file or CSV
new_name = input() #the new name for the converted data

#actual output
format_converter(desired_function, desired_data, new_name)


csv
milk
price
Converted milk


3. SQL database

In [14]:
#using the csv_to_sql function we have made the egg milk database and saved egg milk as a table
format_converter(csv_to_sql,'egg_milk.csv','egg_milk_sql')


Converted egg_milk.csv
